In [ ]:
%pip install geopandas matplotlib plotly nbformat notebook ipywidgets ipyleaflet

In [7]:
import geopandas as gpd
import pandas as pd
from ipyleaflet import Map, GeoJSON, WidgetControl
import ipywidgets as widgets

# Load shapefile
world = gpd.read_file("./data/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp")

# Filter Europe
europe = world[world['CONTINENT'] == 'Europe']

# CO₂ emissions
emission_data = {
    'ADMIN': [
        'Germany', 'France', 'United Kingdom', 'Italy', 'Spain',
        'Poland', 'Netherlands', 'Belgium', 'Sweden', 'Austria'
    ],
    'co2_2019': [750, 340, 380, 320, 270, 320, 170, 120, 45, 65],
    'co2_2021': [670, 310, 340, 300, 250, 290, 160, 110, 44, 63]
}
emissions_df = pd.DataFrame(emission_data)
europe_emissions = europe.merge(emissions_df, how='left', on='ADMIN').fillna(0)

def get_geojson(year):
    def style_callback(feature):
        admin = feature['properties']['ADMIN']
        value = emissions_df.loc[emissions_df['ADMIN'] == admin, f'co2_{year}'].values
        emission = value[0] if len(value) else 0
        intensity = min(emission / 750, 1.0)
        color = f"rgba(255, 0, 0, {intensity})"
        return {
            'fillColor': color,
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.6,
        }

    return GeoJSON(
        data=europe_emissions.__geo_interface__,
        style_callback=style_callback,
        name=f"CO₂ {year}"
    )


# Map setup
center = [54.0, 15.0]
m = Map(center=center, zoom=4)

# Initial GeoJSON layer
layer = get_geojson("2019")
m.add_layer(layer)

# Dropdown widget to switch year
dropdown = widgets.Dropdown(
    options=[('2019 (Pre-Pandemic)', '2019'), ('2021 (Post-Pandemic)', '2021')],
    value='2019',
    description='Year:',
)

# Initial GeoJSON layer in a mutable container
layer_ref = [get_geojson("2019")]
m.add_layer(layer_ref[0])

# Dropdown callback
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        m.remove_layer(layer_ref[0])
        new_layer = get_geojson(change['new'])
        m.add_layer(new_layer)
        layer_ref[0] = new_layer


dropdown.observe(on_change)

control = WidgetControl(widget=dropdown, position='topright')
m.add_control(control)

m


/var/folders/7z/jv8k1cd909v27l8ndwlg06g80000gn/T/ipykernel_97654/517191505.py:22: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  europe_emissions = europe.merge(emissions_df, how='left', on='ADMIN').fillna(0)


Map(center=[54.0, 15.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…